# Garmin Daily Extract (1 Row Per Day)

This notebook authenticates with Garmin Connect, pulls daily history, and builds one row per day with the requested metrics.

In [36]:
# Install once if needed
# %pip install garminconnect python-dotenv pandas

In [ ]:
import os
import time
from datetime import date, datetime, timedelta, timezone
from getpass import getpass
from typing import Any, Dict, Iterator, List, Optional

import pandas as pd
pd.set_option("display.max_columns", None)
from dotenv import load_dotenv
from garminconnect import Garmin

OUTPUT_CSV_PATH = "garmin_daily_metrics.csv"
EARLIEST_DATE_FALLBACK = "2025-12-12"
REQUEST_PAUSE_SECONDS = 0.15


COLUMNS = [
    "date",
    "sleep_duration",
    "sleep_start_time",
    "sleep_end_time",
    "sleep_score",
    "sleep_stage_deep",
    "sleep_stage_light",
    "sleep_stage_rem",
    "sleep_stage_awake",
    "resting_heart_rate",
    "hrv_night_avg",
    "body_battery_min",
    "body_battery_max",
    "body_battery_start",
    "stress_avg",
    "stress_max",
    "steps",
    "intensity_minutes",
    "total_calories",
    "spo2_avg",
    "respiration_avg",
    "weight",
    "body_fat_percentage",
]

In [38]:
def login_garmin() -> Garmin:
    load_dotenv()
    email = os.getenv("GARMIN_EMAIL")
    password = os.getenv("GARMIN_PASSWORD")

    if not email:
        email = input("Garmin email: " ).strip()
    if not password:
        password = getpass("Garmin password: " )

    api = Garmin(email, password)
    api.login()
    print("Authenticated with Garmin Connect.")
    return api

In [39]:
def daterange(start: date, end: date) -> Iterator[date]:
    current = start
    while current <= end:
        yield current
        current += timedelta(days=1)


def safe_call(fn, *args, **kwargs):
    try:
        return fn(*args, **kwargs)
    except Exception as exc:
        print(f"WARN: {fn.__name__}{args} failed: {exc}")
        return None


def get_in(obj: Any, path: tuple) -> Any:
    cur = obj
    for key in path:
        if isinstance(key, int):
            if not isinstance(cur, list) or key >= len(cur):
                return None
            cur = cur[key]
        else:
            if not isinstance(cur, dict) or key not in cur:
                return None
            cur = cur[key]
    return cur


def pick_first(obj: Any, paths: List[tuple], default: Any = None) -> Any:
    for path in paths:
        value = get_in(obj, path)
        if value is not None:
            return value
    return default


def to_iso_from_epoch_millis(value: Any) -> Optional[str]:
    if value is None:
        return None
    try:
        value_int = int(value)
    except (TypeError, ValueError):
        return None
    return datetime.fromtimestamp(value_int / 1000, tz=timezone.utc).isoformat()


def to_date(value: Any) -> Optional[date]:
    if value is None:
        return None
    if isinstance(value, date):
        return value
    if isinstance(value, datetime):
        return value.date()
    if isinstance(value, str):
        try:
            return datetime.fromisoformat(value.replace("Z", "+00:00")).date()
        except ValueError:
            return None
    return None


def first_numeric(values: List[Any]) -> Optional[float]:
    for val in values:
        if isinstance(val, (int, float)):
            return float(val)
    return None


def select_first_entry_of_day(entries: List[Dict[str, Any]]) -> Optional[Dict[str, Any]]:
    if not entries:
        return None

    def _ts(entry: Dict[str, Any]) -> int:
        for key in ("samplePk", "calendarDate", "date", "timestampGMT", "timestampLocal", "measurementTimeGMT", "measurementTimeLocal"):
            val = entry.get(key)
            if isinstance(val, (int, float)):
                return int(val)
            if isinstance(val, str):
                try:
                    return int(datetime.fromisoformat(val.replace("Z", "+00:00")).timestamp())
                except ValueError:
                    continue
        return 0

    return sorted(entries, key=_ts)[0]


def payload_has_data(payload: Any) -> bool:
    if payload is None:
        return False
    if isinstance(payload, list):
        return len(payload) > 0
    if isinstance(payload, dict):
        if not payload:
            return False
        if payload.get("message") and payload.get("message") != "":
            return False
        for val in payload.values():
            if isinstance(val, (int, float)) and val != 0:
                return True
            if isinstance(val, str) and val.strip() != "":
                return True
            if isinstance(val, (list, dict)) and payload_has_data(val):
                return True
        return False
    return False


def discover_start_date(api: Garmin) -> date:
    fallback = date.fromisoformat(EARLIEST_DATE_FALLBACK)
    today = date.today()

    probe_day = fallback
    while probe_day <= today:
        day_s = probe_day.isoformat()
        probes = [
            safe_call(api.get_stats, day_s),
            safe_call(api.get_sleep_data, day_s),
            safe_call(api.get_body_composition, day_s, day_s),
        ]
        if any(payload_has_data(p) for p in probes):
            window_start = max(fallback, probe_day - timedelta(days=31))
            for d in daterange(window_start, probe_day):
                d_s = d.isoformat()
                probes_fine = [
                    safe_call(api.get_stats, d_s),
                    safe_call(api.get_sleep_data, d_s),
                    safe_call(api.get_body_composition, d_s, d_s),
                ]
                if any(payload_has_data(p) for p in probes_fine):
                    return d
            return probe_day
        probe_day += timedelta(days=30)

    return fallback

In [40]:
def extract_daily_row(api: Garmin, day: date) -> Dict[str, Any]:
    day_s = day.isoformat()
    row: Dict[str, Any] = {col: None for col in COLUMNS}
    row["date"] = day_s

    sleep = safe_call(api.get_sleep_data, day_s) or {}
    sleep_dto = pick_first(sleep, [("dailySleepDTO",), ("sleepDTO",), ("sleep",)], {})

    sleep_start_ms = pick_first(
        sleep_dto,
        [
            ("sleepStartTimestampGMT",),
            ("sleepStartTimestampLocal",),
            ("sleepStartTimestamp",),
        ],
    )
    sleep_end_ms = pick_first(
        sleep_dto,
        [
            ("sleepEndTimestampGMT",),
            ("sleepEndTimestampLocal",),
            ("sleepEndTimestamp",),
        ],
    )

    sleep_start_iso = to_iso_from_epoch_millis(sleep_start_ms)
    sleep_end_iso = to_iso_from_epoch_millis(sleep_end_ms)

    sleep_wake_date = to_date(sleep_end_iso)
    if sleep_wake_date == day:
        sleep_duration = pick_first(
            sleep_dto,
            [
                ("sleepTimeSeconds",),
                ("actualSleepSeconds",),
                ("sleepTimeMillis",),
            ],
        )
        if isinstance(sleep_duration, (int, float)) and sleep_duration > 100000:
            sleep_duration = int(sleep_duration) / 1000

        sleep_score = pick_first(
            sleep_dto,
            [
                ("sleepScores", "overall"),
                ("sleepScores", "overallScore"),
                ("sleepScore",),
            ],
        )

        row["sleep_duration"] = sleep_duration
        row["sleep_start_time"] = sleep_start_iso
        row["sleep_end_time"] = sleep_end_iso
        row["sleep_score"] = sleep_score

        row["sleep_stage_deep"] = first_numeric([
            pick_first(sleep_dto, [("deepSleepSeconds",), ("sleepStages", "deep"), ("sleepLevelsMap", "deep")]),
        ])
        row["sleep_stage_light"] = first_numeric([
            pick_first(sleep_dto, [("lightSleepSeconds",), ("sleepStages", "light"), ("sleepLevelsMap", "light")]),
        ])
        row["sleep_stage_rem"] = first_numeric([
            pick_first(sleep_dto, [("remSleepSeconds",), ("sleepStages", "rem"), ("sleepLevelsMap", "rem")]),
        ])
        row["sleep_stage_awake"] = first_numeric([
            pick_first(sleep_dto, [("awakeSleepSeconds",), ("sleepStages", "awake"), ("sleepLevelsMap", "awake")]),
        ])

    rhr = safe_call(api.get_rhr_day, day_s) or {}
    row["resting_heart_rate"] = pick_first(
        rhr,
        [
            ("allMetrics", 0, "value"),
            ("value",),
            ("todayRestingHeartRate",),
            ("restingHeartRate",),
        ],
    )

    hrv = safe_call(api.get_hrv_data, day_s) or {}
    row["hrv_night_avg"] = pick_first(
        hrv,
        [
            ("hrvSummary", "lastNightAvg"),
            ("lastNightAvg",),
            ("nightlyAvg",),
        ],
    )

    body_battery = safe_call(api.get_body_battery, day_s)
    bb_day = body_battery[0] if isinstance(body_battery, list) and body_battery else (body_battery or {})
    bb_values = bb_day.get("bodyBatteryValuesArray") if isinstance(bb_day, dict) else None
    bb_levels = [p[1] for p in bb_values if isinstance(p, list) and len(p) > 1 and isinstance(p[1], (int, float))] if isinstance(bb_values, list) else []
    if bb_levels:
        row["body_battery_min"] = min(bb_levels)
        row["body_battery_max"] = max(bb_levels)
        row["body_battery_start"] = bb_levels[0]
    else:
        row["body_battery_min"] = pick_first(bb_day, [("bodyBatteryLowestValue",), ("dailyMin",), ("min",)])
        row["body_battery_max"] = pick_first(bb_day, [("bodyBatteryHighestValue",), ("dailyMax",), ("max",)])
        row["body_battery_start"] = pick_first(bb_day, [("bodyBatteryStartValue",), ("startValue",), ("start",)])

    stress = safe_call(api.get_stress_data, day_s) or {}
    row["stress_avg"] = pick_first(stress, [("overallStressLevel",), ("avgStressLevel",), ("averageStressLevel",)])
    row["stress_max"] = pick_first(stress, [("maxStressLevel",), ("highestStressLevel",), ("dailyMax",)])

    stats = safe_call(api.get_stats, day_s) or {}
    row["steps"] = pick_first(stats, [("totalSteps",), ("steps",)])
    row["intensity_minutes"] = pick_first(
        stats,
        [
            ("moderateIntensityMinutes",),
            ("activeMinutes",),
            ("totalIntensityMinutes",),
        ],
    )
    if row["intensity_minutes"] is None:
        mod = pick_first(stats, [("moderateIntensityMinutes",)], 0) or 0
        vig = pick_first(stats, [("vigorousIntensityMinutes",)], 0) or 0
        if mod or vig:
            row["intensity_minutes"] = mod + vig

    row["total_calories"] = pick_first(stats, [("totalKilocalories",), ("calories",)])

    spo2 = safe_call(api.get_spo2_data, day_s) or {}
    row["spo2_avg"] = pick_first(
        spo2,
        [
            ("avgSpo2",),
            ("spo2Summary", "averageValue"),
            ("summary", "average"),
        ],
    )

    respiration = safe_call(api.get_respiration_data, day_s) or {}
    row["respiration_avg"] = pick_first(
        respiration,
        [
            ("avgWakingRespirationValue",),
            ("respirationSummary", "avgWakingRespirationValue"),
            ("summary", "averageRespiration"),
        ],
    )

    body_comp = safe_call(api.get_body_composition, day_s, day_s) or {}
    weight_entries = pick_first(
        body_comp,
        [
            ("dateWeightList",),
            ("dailyWeightSummaries",),
            ("weights",),
        ],
        [],
    )
    if isinstance(weight_entries, list):
        first_entry = select_first_entry_of_day(weight_entries)
        if first_entry:
            row["weight"] = pick_first(first_entry, [("weight",), ("weightValue",), ("weightInGrams",)])
            row["body_fat_percentage"] = pick_first(
                first_entry,
                [("bodyFat",), ("bodyFatPercentage",), ("percentFat",)],
            )

    return row

In [41]:
def build_daily_dataset(api: Garmin, start: date, end: date) -> pd.DataFrame:
    rows: List[Dict[str, Any]] = []
    total_days = (end - start).days + 1

    for idx, day in enumerate(daterange(start, end), start=1):
        rows.append(extract_daily_row(api, day))
        if REQUEST_PAUSE_SECONDS > 0:
            time.sleep(REQUEST_PAUSE_SECONDS)
        if idx % 30 == 0 or idx == total_days:
            print(f"Progress: {idx}/{total_days} days")

    df = pd.DataFrame(rows)
    for col in COLUMNS:
        if col not in df.columns:
            df[col] = None
    df = df[COLUMNS]
    df["date"] = pd.to_datetime(df["date"]).dt.strftime("%Y-%m-%d")
    return df

In [42]:
api = login_garmin()
start_date = discover_start_date(api)
end_date = date.today()

print(f"Extract range: {start_date} -> {end_date}")
daily_df = build_daily_dataset(api, start_date, end_date)
daily_df.to_csv(OUTPUT_CSV_PATH, index=False)

print(f"Saved CSV: {OUTPUT_CSV_PATH}")
print(f"Rows: {len(daily_df)}")
print(f"Date min/max: {daily_df['date'].min()} -> {daily_df['date'].max()}")
daily_df.head()

Authenticated with Garmin Connect.
Extract range: 2025-12-10 -> 2026-02-16
Progress: 30/69 days
Progress: 60/69 days
Progress: 69/69 days
Saved CSV: garmin_daily_metrics.csv
Rows: 69
Date min/max: 2025-12-10 -> 2026-02-16


,date,sleep_duration,sleep_start_time,sleep_end_time,sleep_score,sleep_stage_deep,sleep_stage_light,sleep_stage_rem,sleep_stage_awake,resting_heart_rate,hrv_night_avg,body_battery_min,body_battery_max,body_battery_start,stress_avg,stress_max,steps,intensity_minutes,total_calories,spo2_avg,respiration_avg,weight,body_fat_percentage
0,2025-12-10,NaN,None,None,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
1,2025-12-11,NaN,None,None,None,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,None
2,2025-12-12,NaN,None,None,None,NaN,NaN,NaN,NaN,None,NaN,49.0,56.0,56.0,41.0,86.0,192.0,0.0,2229.0,None,14.0,75976.0,None
3,2025-12-13,24900.0,2025-12-13T00:42:41+00:00,2025-12-13T07:43:41+00:00,"{'value': 83, 'qualifierKey': 'GOOD'}",4860.0,15480.0,4560.0,360.0,None,46.0,8.0,98.0,47.0,48.0,99.0,3561.0,0.0,3550.0,None,14.0,NaN,None
4,2025-12-14,11880.0,2025-12-14T08:56:29+00:00,2025-12-14T12:33:29+00:00,"{'value': 38, 'qualifierKey': 'POOR'}",3240.0,8640.0,0.0,1140.0,None,38.0,5.0,54.0,5.0,44.0,99.0,2406.0,0.0,2966.0,None,14.0,NaN,None


In [43]:
# Optional QA checks
assert list(daily_df.columns) == COLUMNS, "Output schema mismatch"

dates_series = pd.to_datetime(daily_df["date"]).sort_values().reset_index(drop=True)
expected_days = (dates_series.iloc[-1] - dates_series.iloc[0]).days + 1 if len(dates_series) else 0
assert len(daily_df) == expected_days, "Expected one row per day without gaps"

print("QA checks passed.")

QA checks passed.


In [47]:
daily_df.tail()

,date,sleep_duration,sleep_start_time,sleep_end_time,sleep_score,sleep_stage_deep,sleep_stage_light,sleep_stage_rem,sleep_stage_awake,resting_heart_rate,hrv_night_avg,body_battery_min,body_battery_max,body_battery_start,stress_avg,stress_max,steps,intensity_minutes,total_calories,spo2_avg,respiration_avg,weight,body_fat_percentage
64,2026-02-12,27180.0,2026-02-11T23:29:09+00:00,2026-02-12T07:12:09+00:00,"{'value': 85, 'qualifierKey': 'GOOD'}",5160.0,16080.0,5940.0,600.0,None,54.0,5.0,91.0,10.0,48.0,99.0,5006.0,0.0,2884.0,None,15.0,NaN,None
65,2026-02-13,28440.0,2026-02-13T01:01:10+00:00,2026-02-13T09:12:10+00:00,"{'value': 69, 'qualifierKey': 'FAIR'}",5040.0,17700.0,5700.0,1020.0,None,33.0,5.0,30.0,5.0,42.0,96.0,9036.0,47.0,2815.0,None,15.0,NaN,None
66,2026-02-14,27258.0,2026-02-14T00:20:17+00:00,2026-02-14T08:05:35+00:00,"{'value': 81, 'qualifierKey': 'GOOD'}",3660.0,19980.0,3660.0,660.0,None,52.0,5.0,70.0,5.0,33.0,98.0,5637.0,25.0,2580.0,None,14.0,NaN,None
67,2026-02-15,30900.0,2026-02-15T01:08:26+00:00,2026-02-15T10:16:26+00:00,"{'value': 89, 'qualifierKey': 'GOOD'}",6240.0,18060.0,6600.0,1980.0,None,64.0,13.0,100.0,13.0,33.0,99.0,10043.0,7.0,2858.0,None,15.0,NaN,None
68,2026-02-16,27120.0,2026-02-15T23:34:49+00:00,2026-02-16T07:17:49+00:00,"{'value': 88, 'qualifierKey': 'GOOD'}",7140.0,13980.0,6000.0,660.0,None,50.0,17.0,77.0,17.0,38.0,95.0,8172.0,25.0,2328.0,None,14.0,NaN,None
